# Möbius‑Digit‑Thread Grid Explorer (v14) — real‑safe damping

**Purpose**  
Rebuilds the full pipeline from scratch so that fractional powers stay **real** and every dial we agreed on is explicit and Colab‑ready.

| knob | value |
|------|-------|
| bright tail | 400 ones |
| gap construction | prime gaps, `gap_scale = 20` |
| spacing | every 3rd dark pulse |
| tiny exponent k | 4.0 (α⁻¹ / 10^k) |
| φ‑micro grid | 0.003944 ± 1 × 10⁻⁶ (9 pts) |
| λ‑sweep | None (=0), 0.2, 0.3, 0.4, 0.5 |
| bit seeds | 64, 128, 256 |

Diagnostics printed for each run:

* digit‑thread finder (π, e, γ, √2) ≥4 consecutive digits  
* power‑of‑3 proximity (9/27/81/729) within 0.5 %  
* Fibonacci closeness check (Δx < 1 × 10⁻⁵ to 64‑ and 128‑bit seeds)

Grid‑search prints the **best 20 rows** sorted by `rel_err`.  
If any row drops **below 1 %** it prints a banner; if it beats **0.1 %** it shouts.

Run the notebook in Colab, tweak `k_vals`, `phi_vals`, `lam_vals` if you want a wider or finer scan.


In [7]:
import mpmath as mp, pandas as pd, itertools, math
mp.mp.dps = 260  # high precision

ALPHA_INV = mp.mpf('137.035999084')  # CODATA 2022
ALPHA   = 1/ALPHA_INV
PHI = (1+mp.sqrt(5))/2


In [8]:
# ---------- utilities ----------
def digits_of(num, n=200):
    s = str(num).replace('.', '')[:n]
    return s

CONST = {
    'pi'   : digits_of(mp.pi),
    'e'    : digits_of(mp.e),
    'phi'  : digits_of(PHI),
    'gamma': digits_of(mp.euler),
    'sqrt2': digits_of(mp.sqrt(2)),
}

def const_thread(xdig):
    """Return first constant tag that shares ≥4 consecutive digits with *xdig*."""
    for name, bank in CONST.items():
        for i in range(len(xdig)-3):
            if xdig[i:i+4] in bank:
                return f'{name}:{xdig[i:i+4]}'
    return '-'

def fib_word_bits(n):
    a,b='0','1'
    while len(b)<n:
        a,b=b,a+b
    return b[:n]

def fib_fraction(bits):
    s=mp.mpf(0)
    for i,b in enumerate(bits):
        if b=='1':
            s += mp.power(2,-(i+1))
    return s

F64  = fib_fraction(fib_word_bits(64))
F128 = fib_fraction(fib_word_bits(128))


In [9]:
# ---------- prime gaps & CF tail ----------
def primes_upto(n=30000):
    sieve=[True]*(n+1); sieve[0:2]=[False,False]
    for i in range(2,int(mp.sqrt(n))+1):
        if sieve[i]:
            sieve[i*i:n+1:i]=[False]*len(range(i*i,n+1,i))
    return [v for v in range(n+1) if sieve[v]]

PRIMES = primes_upto()

def prime_gaps(cnt):
    return [PRIMES[i+1]-PRIMES[i] for i in range(cnt)]

def build_tail(bright_len=400, spacing=3, gap_scale=20):
    gaps=[round(g/ gap_scale,4) for g in prime_gaps(bright_len//spacing)]
    tail=[1] # Start with a number, not a string
    gi=iter(gaps)
    for i in range(bright_len):
        tail.append(1) # Append numbers, not strings
        if (i+1)%spacing==0:
            tail.append(next(gi,1))
    return tail

def cf_to_real(coeffs):
    v=mp.mpf(0)
    for a in reversed(coeffs):
        v=1/(a+v)
    return v

In [10]:
# ---------- Möbius maps ----------
def breath_once(x, tiny, phi):
    x = phi*x/tiny          # φ-scale reciprocal
    x -= tiny               # α injection (tiny = α⁻¹/10ᵏ)
    x = 1/x/9               # Möbius neutraliser
    return x                # reciprocal restores sign

def damped_breath_real(x, tiny, phi, lam=None):
    """Apply one Möbius breath, then x -> x^λ with λ in (0,1]. If lam is None or 0, no damping."""
    x2 = breath_once(x, tiny, phi)
    if not lam:
        return x2
    sign = mp.sign(x2)
    return sign * mp.power(abs(x2), lam)  # stays real


In [11]:
# ---------- grid search ----------
phi_centre = mp.mpf('0.003944')
phi_vals = [phi_centre + 1e-6*i for i in range(-4,5)]  # 9‑pt grid
k_vals  = [4.0]               # start with k = 4.0 ; tweak later
lam_vals = [None,0.2,0.3,0.4,0.5]

tail   = build_tail(spacing=3, gap_scale=20)
seed_c = cf_to_real(tail)

rows=[]
for bits in (64,128,256):
    for k in k_vals:
        tiny = ALPHA_INV/mp.power(10,k)
        for phi in phi_vals:
            for lam in lam_vals:
                x2 = damped_breath_real(seed_c+tiny, tiny, phi, lam)
                rel = float(abs((x2-ALPHA)/ALPHA))
                rows.append(dict(bits=bits,k=k,phi=float(phi),lam=lam or 0,
                                 rel_err=rel,
                                 thread=const_thread(str(x2)[:402]),
                                 fib64_near=abs(x2-F64)<1e-5,
                                 fib128_near=abs(x2-F128)<1e-5))
df = pd.DataFrame(rows).sort_values('rel_err')
display(df.head(20))

best = df.iloc[0]
if best.rel_err < 0.01:
    print(f"\n*** Hit <1 %! bits={best.bits} k={best.k} phi={best.phi} lam={best.lam} rel={best.rel_err:.6g} ***")
if best.rel_err < 0.001:
    print("\n***** Hit <0.1 %! breakthrough! *****")


,bits,k,phi,lam,rel_err,thread,fib64_near,fib128_near
40,64,4.0,0.003948,0.0,87.298972,pi:1027,False,False
85,128,4.0,0.003948,0.0,87.298972,pi:1027,False,False
130,256,4.0,0.003948,0.0,87.298972,pi:1027,False,False
80,128,4.0,0.003947,0.0,87.323122,pi:8128,False,False
125,256,4.0,0.003947,0.0,87.323122,pi:8128,False,False
35,64,4.0,0.003947,0.0,87.323122,pi:8128,False,False
30,64,4.0,0.003946,0.0,87.347284,pi:3852,False,False
75,128,4.0,0.003946,0.0,87.347284,pi:3852,False,False
120,256,4.0,0.003946,0.0,87.347284,pi:3852,False,False
115,256,4.0,0.003945,0.0,87.371460,pi:2948,False,False
